In [35]:
import numpy as np
import pandas as pd
import re
import pickle
import os
from IPython.core.debugger import set_trace


# Read text

In [2]:
def cleantxt(fname):
    rawname = fname+'_raw'
    if fname[-3:]=='txt':
        rawname = fname[:-4]+'_raw.txt'
    
    with open(fname, 'r') as f, open(rawname, 'a') as rawf:
        content = f.readlines()
        for line in f:
            rawf.write(line)
            
    # remove whitespace at the end. keep the tab \t to mark level of question    
    content = [x.strip('\n') for x in content] 
   
    for line in content:
        if line[0]==' ': # transform white spaces into tab which is used to count levels
            line = line.replace('  ','\t') 
        if line.find('？')>0: # deal with chinese characters
            line = line.replace('？','?')
        if line.find('。')>0:
            line = line.replace('。','.')
        with open(fname, 'w') as wf: # rewrite the original text name so nothing weird in the processed textfile.
            wf.write(line+'\n')
    return content,cleanname
        

# Text to quesnet
For paper notes, the first line will be the title of this paper, which should be linked to the paper data structure.




In [83]:
def fun_ntype(edge):
    if edge=='a' or edge =='answer':
        nid = 'a%d'%np.random.randint(100000)
        nftr = 'Answer'
    else: #including unspecified edge feature
        nid = 'q%d'%np.random.randint(100000)
        nftr = 'Question'
    return nid,nftr
def getedge(line):
    # get edge
    if line.find('...')>0: # edge of a group
        ed = line[line.find('_')+1:line.find('...')]
    else: # edge of single node
        ed = line[line.find('_')+1:line.rfind('_')]
    if ed == 'a' or ed.lower()=='answer':
        ed = 'answer' # rename the txt shorthand
    elif ed == 'specify':
        ed = 'specification'
    # get speaker
    if line.find('@')>0: # speaker other than this paper -- usually me
        print(line.find(' '))
        print(line)
        set_trace()
        speaker = line[line.find('@')+1:line.find(' ')]
    else:
        speaker = ''
    return ed,speaker
        
def node2label(node):
    if node.find('"')>0:
        label = node[:node.find('"')]
        content = node[node.find('"'):]
    else:
        label = node
        content=""
    return label,content

In [84]:
def txt2quesnet(outdir,fin='',nodedist={},alledge=[]):
    for key in ['label','type','id','content','speaker']:
        nodedist[key]=[]
    with open(fin,'r') as f:
        content = f.readlines()
    
    # create a paper entity
    thisp = {'id':'Ep%d'%np.random.randint(100000)} #E=entity,p=paper
    #FUTURE: may use other ways to record the paper info
    fname = fin[fin.rfind('/')+1:] # get rid of the file directory 
    thisp['label']=fname.strip('.txt')
    for ks,s in enumerate(fname):
        if s.isdigit:
            thisp['author'] = fname[:ks]
            break
    thisp['year']=fname[ks:ks+4]
    # if the first line is the paper name
    if content[0]=='#':
        thisp['title']=content[0][3:]
        
    # create multiple question threads from each root
    maxlevel = 10 # maximum 10 levels down the tree
    allngrps=[]
    for line in content[1:]:
        if len(line.strip())<1: # skip empty lines
            continue
        elif line[:2]=='##':#ending sign
            break
        else: # a valid new line
            line = line.rstrip()
            if line[0].isalpha(): # a root node doesn't start with tab
                    rootlv=0
                    rootlist = [[] for k in range(maxlevel)]# restart a new root list
                    nid = 'q%d'%np.random.randint(100000) # Root is by default a question -- always start with Q!
                    rootlist[0]=nid
                    node = line
                    edge = ''
                    spkr = ''
            elif line.find('...')>0:
                if line.find('_')>0:
                    nodegrp = [] # for reading parallel nodes
                    edge,spkr = getedge(line)                    
                    continue
                else:
                    node = line[line.rfind('...')+4:]                    
                    nid,nftr = fun_ntype(edge) # edge is inherented from the group head
                    nodegrp.append(nid)
            elif line.find('end')>0:
                allngrps.append(nodegrp)
                continue                
            elif line.find('_')>0: # a normal edge
                edge,spkr = getedge(line)
                nid,nftr = fun_ntype(edge)                
                node = line[line.rfind('_')+2:]
            else:
                print(line)
                print("can't recognize your grammar...")                        
            level = int((len(line) - len(line.strip('\t')))/len('\t')) # an absolute level
            # Construct the edge dictionary        
            rootlist[level]=nid            
            if level>0: # note a root node, add an edge
                if len(rootlist[level-1])==0:
                    print(node)
                    set_trace()
                edgedict = {'start':rootlist[level-1],'end':nid,'label':edge,'id':'l%d'%np.random.randint(100000)}
                alledge.append(edgedict)                          
            # Construct the node dictionary    
            label,content=node2label(node)
            if label not in nodedist['label']: 
                if edge == 'answer':
                    nodedist['type'].append('Answer')    
                else:
                    if level==0:
                        nodedist['type'].append('Root question')
                    else:
                        nodedist['type'].append('Question')    
                nodedist['label'].append(label)
                nodedist['id'].append(nid)
                nodedist['content'].append(content)
                #nodedist['reference'].append(reference)
                if len(spkr)==0:
                    spkr = thisp['id'] # if not specified, default speaker is this paper
                nodedist['speaker'].append(spkr)
            else: # this label already exists, just adding new edge here
                nid = nodedist['id'][nodedist['label'].index(label)]
                
            
    nodedf = pd.DataFrame(nodedist)
    edgedf = pd.DataFrame(alledge)  
    #os.makedirs(filedir)
 
    pickle.dump([nodedf,edgedf,allngrps],open(outdir+thisp['label']+'_qnet.p','bw'))
    print('generated database in:\n'+outdir+thisp['label']+'_qnet.p')
    print(nodedist['speaker'])
    return thisp # output the paper entity. TODO: other entities mentioned in the nodes should also be added, so that return a full database

In [85]:
fname='../txtnotes/papernotes/Pathak2019_Self-Supervised.txt'
outdir='../database/curiosity/algorithms/'
Epaper=txt2quesnet(outdir,fname) 

generated database in:
../database/curiosity/algorithms/Pathak2019_Self-Supervised_qnet.p
['Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717', 'Ep50717']


In [86]:
[nodedf,edgedf,allngrps] = pickle.load(open('../database/curiosity/algorithms/Pathak2019_Self-Supervised_qnet.p','rb'))
edgedf

,end,id,label,start
0,a57287,l31214,answer,q95465
1,q44673,l99790,specification,a57287
2,a10584,l6014,answer,q44673
3,a82825,l13062,answer,a57287
4,q35245,l95387,specification,a82825
5,a11922,l44906,answer,q35245
6,q60818,l71970,,a11922
7,q63327,l31447,comment,q60818
8,q82021,l73876,comment,a82825
9,a15595,l9230,answer,q82021


next step: use "pickle to networkx" notebook to do the visualization